In [106]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
html_text = requests.get("https://www.espncricinfo.com/team")
soup = BeautifulSoup(html_text.content,"html.parser")

# Extract Teams name and flags

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

# Navigate to the website containing the images
driver.get("https://www.espncricinfo.com/team")

# Wait for the image elements to appear on the page
wait = WebDriverWait(driver, 10)
images = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))

# Extract the src attribute from each image element
team_image_urls = [img.get_attribute("src") for img in images]
team_image_urls = team_image_urls[:12]
# Print the list of image URLs
print(team_image_urls)

# Close the browser
driver.quit()

['https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/321000/321005.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/340400/340493.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/341400/341456.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313114.logo.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313128.logo.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313149.logo.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/340500/340505.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313129.logo.png', 'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313125.logo.png', 'https://img1.hscicdn.com/image/upload

In [23]:
team_name_span = soup.find_all("span",class_ = "ds-text-title-s ds-font-bold")
team_names = [name.text for name in team_name_span]
team_names = team_names[:12]
print(team_names)

['Afghanistan', 'Australia', 'Bangladesh', 'England', 'India', 'Ireland', 'New Zealand', 'Pakistan', 'South Africa', 'Sri Lanka', 'West Indies', 'Zimbabwe']


In [27]:
import csv
fields = ['Team Name','Team Image']
team_info_list = zip(team_names,team_image_urls)
with open('team_info.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(fields)
    writer.writerows(team_info_list)

# Extract teams world cup performance

In [48]:
records_text = requests.get("https://stats.espncricinfo.com/ci/engine/records/team/results_summary.html?id=12;type=trophy")
records_soup = BeautifulSoup(records_text.content,"html.parser")
table = records_soup.find("table",class_ = "engineTable")
thead = [th.text for th in table.find_all("th")]
rows = []
for tr in table.find_all("tr"):
    row = [td.text for td in tr.find_all("td")]
    if row:
        rows.append(row)
rows

[['Afghanistan', '2015-2019', '15', '1', '14', '0', '0', '6.66'],
 ['Australia', '1975-2019', '94', '69', '23', '1', '1', '74.73'],
 ['Bangladesh', '1999-2019', '40', '14', '25', '0', '1', '35.89'],
 ['Bermuda', '2007-2007', '3', '0', '3', '0', '0', '0'],
 ['Canada', '1979-2011', '18', '2', '16', '0', '0', '11.11'],
 ['East Africa', '1975-1975', '3', '0', '3', '0', '0', '0'],
 ['England', '1975-2019', '83', '48', '32', '2', '1', '59.75'],
 ['India', '1975-2019', '84', '53', '29', '1', '1', '64.45'],
 ['Ireland', '2007-2015', '21', '7', '13', '1', '0', '35.71'],
 ['Kenya', '1996-2011', '29', '6', '22', '0', '1', '21.42'],
 ['Namibia', '2003-2003', '6', '0', '6', '0', '0', '0'],
 ['Netherlands', '1996-2011', '20', '2', '18', '0', '0', '10'],
 ['New Zealand', '1975-2019', '89', '54', '33', '1', '1', '61.93'],
 ['Pakistan', '1975-2019', '79', '45', '32', '0', '2', '58.44'],
 ['Scotland', '1999-2015', '14', '0', '14', '0', '0', '0'],
 ['South Africa', '1992-2019', '64', '38', '23', '2', '1'

In [49]:
#store the world_cup data for all teams
with open('world_cup_records.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(thead)
    writer.writerows(rows)

# Merge the teams name and images data with teams world cup performance data

In [68]:
with open('team_info.csv','r') as file:
    reader = csv.reader(file)
    team_data = list(reader)

with open('world_cup_records.csv','r') as file:
    reader = csv.reader(file)
    world_cup_data = list(reader)

combined_data = []
for team in team_data:
    team_name = team[0]
    team_image = team[1]    
    for wc_team in world_cup_data:
        if wc_team[0] == team_name:
            team_span = wc_team[1]
            team_matches = wc_team[2]
            team_matches_won = wc_team[3]
            team_matches_lost = wc_team[4]
            team_matches_tied = wc_team[5]
            team_matches_runRate = wc_team[6]
            team_matches_winningPer = wc_team[7]
            combined_data.append([team_name,team_image,team_span,team_matches,team_matches_won,team_matches_lost,team_matches_tied,team_matches_runRate,team_matches_winningPer])

# Extract the teams world cup winning data

In [72]:
#teams world cup winning data
winning_text = requests.get("https://www.careerpower.in/cricket-world-cup-winners-list.html")
winning_soup = BeautifulSoup(winning_text.content,"html.parser")


In [103]:
table = winning_soup.find_all("table")[1]
heads = [th.text for th in table.find_all("tr")[0]]
thead = [head.replace("\xa0"," ") for head in heads]
winning_data = []
for tr in table.find_all("tr")[1:]:
    row = [td.text for td in tr.find_all("td")]
    winning_data.append(row)

In [102]:
combined_data

[['Afghanistan',
  'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/321000/321005.png',
  '2015-2019',
  '15',
  '1',
  '14',
  '0',
  '0',
  '6.66'],
 ['Australia',
  'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/340400/340493.png',
  '1975-2019',
  '94',
  '69',
  '23',
  '1',
  '1',
  '74.73'],
 ['Bangladesh',
  'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/341400/341456.png',
  '1999-2019',
  '40',
  '14',
  '25',
  '0',
  '1',
  '35.89'],
 ['England',
  'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313114.logo.png',
  '1975-2019',
  '83',
  '48',
  '32',
  '2',
  '1',
  '59.75'],
 ['India',
  'https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_80/lsci/db/PICTURES/CMS/313100/313128.logo.png',
  '1975-2019',
  '84',
  '53',
  '29',
  '1',
  '1',
  '64.45'],
 ['Ireland',
  'https://img1.hscicdn.com/image/upload/f_auto,t_ds

In [104]:
winning_data

[['Australia', '7', '5', '2', '1987, 1999, 2003, 2007, 2015', '1975, 1996'],
 ['England', '4', '1', '3', '2019', '1979, 1987, 1992'],
 ['India', '3', '2', '1', '1983, 2011', '2003'],
 ['New Zealand', '2', '0', '2', '-', '2015, 2019'],
 ['Pakistan', '2', '1', '1', '1992', '1999'],
 ['Sri Lanka', '3', '1', '2', '1996', '2007, 2011'],
 ['West Indies', '3', '2', '1', '1975, 1979', '1983']]

#  Merge all the data to form the final teams_data
            
            
    

In [119]:
df1 = pd.DataFrame(combined_data,columns = ["Team Name","Flag Images","Span","Matches","Won","Lost","Tied","Net Run Rate(NR)","Winning %"])
df2 = pd.DataFrame(winning_data,columns = ["Team Name","Final Apperances","Winners","Runners","Years Won","Years Runners"])

merged_df = pd.merge(df1, df2, on='Team Name', how='left')
merged_df = merged_df.where((pd.notnull(merged_df)), '-')

;